In [ ]:
import sys
import json
import subprocess
import os
from tqdm import tqdm

import numpy as np

from numba import jit, vectorize
import pandas as pd

from typing import Tuple

# Running it live

## Set up the arrays and functions used in advance

Alternatively they can also be set up when "BEGIN" is read

In [ ]:
num_markets = 12001
num_data_points = 20000

volume = np.empty((num_markets * num_data_points), dtype=np.float64) * np.nan
price = np.empty((num_markets * num_data_points), dtype=np.float32) * np.nan
volume_price_product = np.empty((num_markets * num_data_points), dtype=np.float64) * np.nan
is_buy = np.empty((num_markets * num_data_points), dtype=float) * np.nan

market_analysis = np.zeros((num_markets, 5), dtype=np.float64)

## Declare the functions to be used

In [ ]:
def get_data(line: dict) -> Tuple[int, int, np.ndarray]:
    idx = line["id"]
    market = line["market"]
    position_idx = market * num_data_points + idx
    
    volume[position_idx] = line["volume"]
    price[position_idx] = line["price"]
    volume_price_product[position_idx] = line["volume"] * line["price"]
    is_buy[position_idx] = line["is_buy"]
    return idx, market, volume, price, volume_price_product, is_buy

In [ ]:
@jit(nopython=True)
def analyze_data(
    idx: int, 
    market: int, 
    volume: np.ndarray, 
    price: np.ndarray, 
    volume_price_product: np.ndarray, 
    is_buy: np.ndarray, 
    market_analysis: np.ndarray
):
    start_pos_idx = market * num_data_points
    end_pos_idx = market * num_data_points + idx + 1
    
    # Total volume
    market_analysis[market, 0] = np.nansum(volume[start_pos_idx:end_pos_idx])
    # Mean price
    market_analysis[market, 1] = np.nanmean(price[start_pos_idx:end_pos_idx])
    # Mean volume
    market_analysis[market, 2] = np.nanmean(volume[start_pos_idx:end_pos_idx])
    # Volume weighted price mean
    market_analysis[market, 3] = np.nanmean(volume_price_product[start_pos_idx:end_pos_idx])
    # Percentage buys
    not_nan = is_buy[start_pos_idx:end_pos_idx][np.isnan(is_buy[start_pos_idx:end_pos_idx]) == False]
    if not_nan.size > 0:
        market_analysis[market, 4] = np.count_nonzero(not_nan) / not_nan.size

analyze_data(0, 0, volume, price, volume_price_product, is_buy, market_analysis)

In [ ]:
# Create a subprocess to read the stdout of the bin
proc = subprocess.Popen([os.path.join(os.getcwd(), 'stdoutinator_amd64_darwin.bin')], stdout=subprocess.PIPE)

# Currently limiting the range due to memory issues with the full 10M array, 8 GB is too small :(

# while True:
for _ in tqdm(range(num_data_points)):
    line = proc.stdout.readline().decode("utf-8").rstrip()
    
    if not line:
        break
    if line == "BEGIN":
        # Arrays and functions can also be initialized here
        continue
    if line == "END":
        print("Done")
        break
        
    idx, market, volume, price, volume_price_product, is_buy = get_data(line=json.loads(line))
    analyze_data(
        idx=idx, 
        market=market, 
        volume=volume, 
        price=price, 
        volume_price_product=volume_price_product, 
        is_buy=is_buy, 
        market_analysis=market_analysis
    )
            
    # Extract relevant data from the json
    # Note that `id` has been renamed to `idx`to prevent built-in conflicts

In [ ]:
df = pd.DataFrame(columns=["total_volume", "mean_price", "mean_volume", "mean_volume_weighted_price", "percentage_buys"])

for index, element in enumerate(market_analysis):
    df.loc[index] = element

In [ ]:
df